### 깃 저장소 가져오기

In [ ]:
!git clone https://github.com/CBNU-Graduation-Projects/Leaf-Disease-Detection-and-Classification.git


Cloning into 'Leaf-Disease-Detection-and-Classification'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 96 (delta 32), reused 73 (delta 23), pack-reused 0
Receiving objects: 100% (96/96), 21.35 MiB | 11.73 MiB/s, done.
Resolving deltas: 100% (32/32), done.


## GCP 스토리지 버킷 업로드, 다운로드, 데이터 셋 구축

### 버킷과 같은 GCE 인스턴스에서 실행 중이 아닐경우
- Google Cloud SDK를 활용한 인증 필요

In [1]:
!pip install -q google-cloud-storage
!pip install -q google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.2 MB/s eta 0:00:00


In [2]:
from google.colab import auth
auth.authenticate_user()

###  GCP 버킷에서 데이터 다운로드

#### 버킷을 통째로 다운로드

- download_bucket_with_transfer_manager : 버킷에 있는 모든 데이터를 가져온다.
- bucket_name : GCP 버킷 이름
- destination_directory : 받아온 데이터를 저장할 로컬 디렉토리 경로

In [5]:
from google.cloud.storage import Client, transfer_manager
bucket_name = "버킷 이름"
destination_directory = "/content/"

def download_bucket_with_transfer_manager(
    bucket_name, destination_directory="", workers=8, max_results=1000
):
    """Download all of the blobs in a bucket, concurrently in a process pool.

    The filename of each blob once downloaded is derived from the blob name and
    the `destination_directory `parameter. For complete control of the filename
    of each blob, use transfer_manager.download_many() instead.

    Directories will be created automatically as needed, for instance to
    accommodate blob names that include slashes.
    """

    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The directory on your computer to which to download all of the files. This
    # string is prepended (with os.path.join()) to the name of each blob to form
    # the full path. Relative paths and absolute paths are both accepted. An
    # empty string means "the current working directory". Note that this
    # parameter allows accepts directory traversal ("../" etc.) and is not
    # intended for unsanitized end user input.
    # destination_directory = ""

    # The maximum number of processes to use for the operation. The performance
    # impact of this value depends on the use case, but smaller files usually
    # benefit from a higher number of processes. Each additional process occupies
    # some CPU and memory resources until finished. Threads can be used instead
    # of processes by passing `worker_type=transfer_manager.THREAD`.
    # workers=8

    # The maximum number of results to fetch from bucket.list_blobs(). This
    # sample code fetches all of the blobs up to max_results and queues them all
    # for download at once. Though they will still be executed in batches up to
    # the processes limit, queueing them all at once can be taxing on system
    # memory if buckets are very large. Adjust max_results as needed for your
    # system environment, or set it to None if you are sure the bucket is not
    # too large to hold in memory easily.
    # max_results=1000


    storage_client = Client()
    bucket = storage_client.bucket(bucket_name)

    blob_names = [blob.name for blob in bucket.list_blobs(max_results=max_results)]

    results = transfer_manager.download_many_to_path(
        bucket, blob_names, destination_directory=destination_directory, max_workers=workers
    )

    for name, result in zip(blob_names, results):
        # The results list is either `None` or an exception for each blob in
        # the input list, in order.

        if isinstance(result, Exception):
            print("Failed to download {} due to exception: {}".format(name, result))
        else:
            print("Downloaded {} to {}.".format(name, destination_directory + name))

if __name__ == "__main__":
    download_bucket_with_transfer_manager(bucket_name, destination_directory)

KeyboardInterrupt: 

#### 버킷 안의 특정 폴더만 다운로드

In [3]:
def download_bucket_folder_with_transfer_manager(
    bucket_name, folder_name, destination_directory="", workers=8, max_results=1000
):
    """Download blobs from a specific folder in a bucket using transfer manager."""
    from google.cloud.storage import Client, transfer_manager

    storage_client = Client()
    bucket = storage_client.bucket(bucket_name)

    # Get blobs with the given folder prefix
    blob_names = [blob.name for blob in bucket.list_blobs(prefix=folder_name, max_results=max_results)]

    results = transfer_manager.download_many_to_path(
        bucket, blob_names, destination_directory=destination_directory, max_workers=workers
    )

    for name, result in zip(blob_names, results):
        if isinstance(result, Exception):
            print("Failed to download {} due to exception: {}".format(name, result))
        else:
            print("Downloaded {} to {}.".format(name, destination_directory + name))

if __name__ == "__main__":
    bucket_name = "버킷 이름"
    folder_name = "01.데이터(1000)/"
    destination_directory = "/content/"

    download_bucket_folder_with_transfer_manager(
        bucket_name, folder_name, destination_directory
    )


/usr/local/lib/python3.10/dist-packages/google/cloud/storage/transfer_manager.py:30: UserWarning: The module `transfer_manager` is a preview feature. Functionality and API may change. This warning will be removed in a future release.
  warnings.warn(


Downloaded 01.데이터(1000)/1.Training/라벨링데이터/02.고추/0.정상.zip to /content/01.데이터(1000)/1.Training/라벨링데이터/02.고추/0.정상.zip.
Downloaded 01.데이터(1000)/1.Training/라벨링데이터/02.고추/1.질병.zip to /content/01.데이터(1000)/1.Training/라벨링데이터/02.고추/1.질병.zip.
Downloaded 01.데이터(1000)/1.Training/라벨링데이터/05.상추/0.정상.zip to /content/01.데이터(1000)/1.Training/라벨링데이터/05.상추/0.정상.zip.
Downloaded 01.데이터(1000)/1.Training/라벨링데이터/05.상추/1.질병.zip to /content/01.데이터(1000)/1.Training/라벨링데이터/05.상추/1.질병.zip.
Downloaded 01.데이터(1000)/1.Training/라벨링데이터/11.토마토/0.정상.zip to /content/01.데이터(1000)/1.Training/라벨링데이터/11.토마토/0.정상.zip.
Downloaded 01.데이터(1000)/1.Training/라벨링데이터/11.토마토/1.질병.zip to /content/01.데이터(1000)/1.Training/라벨링데이터/11.토마토/1.질병.zip.
Downloaded 01.데이터(1000)/1.Training/원천데이터/02.고추/0.정상.zip to /content/01.데이터(1000)/1.Training/원천데이터/02.고추/0.정상.zip.
Downloaded 01.데이터(1000)/1.Training/원천데이터/02.고추/1.질병.zip to /content/01.데이터(1000)/1.Training/원천데이터/02.고추/1.질병.zip.
Downloaded 01.데이터(1000)/1.Training/원천데이터/05.상추/0.정상.zip to /content/01.데

### 다운로드한 기존 데이터 셋을 축소하여 저장하기
- 기존 데이터 셋은 너무 방대하여 fine tuning 하기 위한 데이터로 사용하기엔 적합하지 않다
- 따라서 기존 데이터를 최대 1000개의 이미지로 폴더를 재구축하여 fine tuning 전용 데이터 셋을 구축한다.

In [ ]:
# 축소된 데이터셋을 구분하기위해 디렉토리 이름 변경
import os

# 이전 디렉토리 이름
old_dir_name = '/content/01.데이터'

# 새로운 디렉토리 이름
new_dir_name = '/content/01.데이터(1000)'

# 디렉토리 이름 변경
os.rename(old_dir_name, new_dir_name)

root_path = "/content/01.데이터(1000)"

In [ ]:
# 기존 데이터셋 압축 해제
import os
import zipfile

def unzip_files_in_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".zip"):

                zip_file_path = os.path.join(root, file)
                extract_dir = os.path.splitext(zip_file_path)[0]  # .zip 확장자 제거

                # 디렉토리가 존재하지 않으면 생성
                if not os.path.exists(extract_dir):
                    os.makedirs(extract_dir)

                # ZIP 파일 해제
                with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_dir)
                print(f"Unzipped {zip_file_path} to {extract_dir}")

                # ZIP 파일 삭제
                os.remove(zip_file_path)
                print(f"Deleted {zip_file_path}")


In [ ]:
# 데이터가 있는 폴더 경로를 찾아주는 함
import os

def find_specific_dirs(root_directory, required_sub_folders, optional_sub_folders):
    matched_dirs = []

    for dirpath, dirnames, filenames in os.walk(root_directory):
        if all(sub_folder in dirpath for sub_folder in required_sub_folders) and \
                any(sub_folder in dirpath for sub_folder in optional_sub_folders):
                full_path = os.path.join(dirpath)
                matched_dirs.append(full_path)

    return matched_dirs

##### 각각의 데이터를 특정 개수만 남기고 삭제

In [ ]:
# 데이터셋 축소를 위해 특정 개수의 파일만 남기고 삭제
import os

def delete_files(folder_path, keep_count=1000):
    # 폴더에 있는 모든 파일을 가져옵니다.
    files = os.listdir(folder_path)

    # 파일을 이름 순서대로 정렬합니다.
    files.sort()

    # 삭제할 파일을 선택합니다. 앞에서 1000개를 제외한 나머지 파일들.
    files_to_delete = files[keep_count:]

    # 파일을 삭제합니다.
    for file in files_to_delete:
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            os.remove(file_path)

    print(f"Deleted: {folder_path}")

def process_multiple_folders(folder_paths, keep_count=1000):
    for folder_path in folder_paths:
        delete_files(folder_path, keep_count)



In [ ]:
folders = find_specific_dirs(root_path, ['01.데이터'], ['0.정상','1.질병'])
process_multiple_folders(folders,1000) #1000개 이상의 데이터를 가지고 있는 폴더는 데이터가 축소된다.

##### 축소시킨 데이터 셋 유효성 검증

In [ ]:
# 파일 이름 비교 함수: 파일 확장자를 제거한 이름을 비교하여 동일한지 확인
def is_same_filename(file1, file2):
  # 파일 확장자를 제거한 파일 이름을 가져옴
    base1 = os.path.basename(file1).split('.')[0]
    base2 = os.path.basename(file2).split('.')[0]

    # 확장자를 제거한 파일 이름이 같은지 비교
    return base1 == base2

# 폴더 내의 모든 파일명을 리스트로 생성
def get_files_in_folder(folder_path):
    return [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# 두 폴더를 비교하여 대응되는 이미지와 라벨 데이터를 확인하고,
# 매핑된 리스트와 각각 매칭되지 않은 파일 리스트를 반환
def pair_common_files(folder1, folder2):
    files1 = get_files_in_folder(folder1)
    files2 = get_files_in_folder(folder2)

    paired_list = []
    unmatched_files1 = []
    unmatched_files2 = files2[:]  # 복사하여 나중에 제거할 때 사용

    for file1 in files1:
        matched = False
        for file2 in files2:
            if is_same_filename(file1, file2): # 파일 이름 비교를 위한 함수 호출
                paired_list.append((os.path.join(folder1, file1), os.path.join(folder2, file2)))
                unmatched_files2.remove(file2)
                matched = True
                break
        if not matched:
            unmatched_files1.append(os.path.join(folder1, file1))

    unmatched_files2 = [os.path.join(folder2, file) for file in unmatched_files2]

    return paired_list, unmatched_files1, unmatched_files2

In [ ]:
import os

def count_files_in_folder(folder_path):
    file_count = 0
    for root, dirs, files in os.walk(folder_path):
        file_count += len(files)
    return file_count

def print_file_counts_in_folders(folder_paths):
    for folder_path in folder_paths:
        file_count = count_files_in_folder(folder_path)
        print(f"Folder: {folder_path}, File count: {file_count}")

In [ ]:
# 각 이미지 폴더와 라벨 폴더를 비교하여 대응되는 파일 검사
import os

image_folder_paths = find_specific_dirs(root_path, ['원천데이터'], ['0.정상','1.질병'])
label_folder_paths = find_specific_dirs(root_path, ['라벨링데이터'], ['0.정상','1.질병'])

for index, image_folder_path in enumerate(image_folder_paths):
  paired_list, unmatched_files1, unmatched_files2 = pair_common_files(
      image_folder_path,
      label_folder_paths[index]
      )
  print("==========================================")
  print("이미지 폴더 경로:", image_folder_path)
  print("라벨링 폴더 경로:", label_folder_paths[index])
  print("매칭되지 않은 이미지 파일 개수:", len(unmatched_files1))
  print("매칭되지 않은 라벨 파일 개수:", len(unmatched_files2))

# 각각의 데이터 폴더들의 개수 출력 -> 데이터 축소 검증
print_file_counts_in_folders(folders)


##### bucket에 저장하기위해 축소시킨 데이터셋을 다시 압축

In [ ]:
import os
import shutil
import zipfile

def zip_directory(source_dir, zip_name):
    """Compress a directory into a ZIP file."""
    shutil.make_archive(zip_name, 'zip', source_dir)
    print(f"Directory {source_dir} compressed to {zip_name}.zip")

def delete_directory(directory):
    """Delete a directory and all its contents."""
    shutil.rmtree(directory)
    print(f"Directory {directory} deleted.")

def compress_and_delete_directory(source_dir):
    """Compress a directory and then delete it."""
    zip_name = os.path.basename(source_dir)  # Get the directory name for the zip file
    parent_dir = os.path.dirname(source_dir)  # Get the parent directory

    # Full path for the zip file
    zip_path = os.path.join(parent_dir, zip_name)

    # Compress the directory
    zip_directory(source_dir, zip_path)

    # Delete the original directory
    delete_directory(source_dir)

In [ ]:
for folder in folders:
  compress_and_delete_directory(folder)

### 버킷에 데이터 업로드하기
  - 업로드시 버킷에 저장될 기본적인 prefix(경로)는 ''이다. -> 이후 경로는 파일 경로를 따른다.
    - 예
      - local_directory_path = "/content/"
      - /content/ex.txt를 저장할때 버킷에 저장되는 prefix는 ex.txt 이다.
      - /content/ex/ex.txt를 저장한다면 버킷에 저장되는 prefix는 ex/ex.txt이다.
  - bucket_name : GCP 버킷 이름
  - local_dir : 업로드할 폴더 경로 로컬 디렉토리 경로


In [ ]:
from google.cloud import storage
import os
from google.cloud.storage import Client

bucket_name = "버킷 이름"
local_directory_path = "/content/"

def upload_directory(bucket_name, source_directory):
    storage_client = Client()
    bucket = storage_client.bucket(bucket_name)

    filenames = []
    for root, _, files in os.walk(source_directory):
        for file in files:
            full_path = os.path.join(root, file)
            destination_blob_name = os.path.relpath(full_path, source_directory)
            filenames.append((full_path, destination_blob_name))

    existing_blobs = {blob.name for blob in bucket.list_blobs(prefix="")}

    skipped_files = []
    uploaded_files = []
    failed_uploads = []

    print(filenames)
    print(existing_blobs)

    for full_path, destination_blob_name in filenames:
        if destination_blob_name in existing_blobs:
            skipped_files.append(full_path)
        else:
            try:
                blob = bucket.blob(destination_blob_name)
                blob.upload_from_filename(full_path)
                uploaded_files.append(full_path)
            except Exception as e:
                failed_uploads.append((full_path, str(e)))

    print("Uploaded files:")
    for file in uploaded_files:
        print(f"Uploaded {file} to {destination_blob_name} in bucket {bucket.name}.")

    print("\nSkipped files:")
    for file in skipped_files:
        print(f"Skipped {file} as it already exists in bucket {bucket.name}.")

    print("\nFailed uploads:")
    for file, error in failed_uploads:
        print(f"Failed to upload {file} due to exception: {error}")

upload_directory(bucket_name, local_directory_path)


[('/content/01.데이터/1.Training/라벨링데이터/11.토마토/1.질병.zip', '01.데이터/1.Training/라벨링데이터/11.토마토/1.질병.zip'), ('/content/01.데이터/1.Training/라벨링데이터/11.토마토/0.정상.zip', '01.데이터/1.Training/라벨링데이터/11.토마토/0.정상.zip'), ('/content/01.데이터/1.Training/라벨링데이터/05.상추/1.질병.zip', '01.데이터/1.Training/라벨링데이터/05.상추/1.질병.zip'), ('/content/01.데이터/1.Training/라벨링데이터/05.상추/0.정상.zip', '01.데이터/1.Training/라벨링데이터/05.상추/0.정상.zip'), ('/content/01.데이터/1.Training/라벨링데이터/02.고추/1.질병.zip', '01.데이터/1.Training/라벨링데이터/02.고추/1.질병.zip'), ('/content/01.데이터/1.Training/라벨링데이터/02.고추/0.정상.zip', '01.데이터/1.Training/라벨링데이터/02.고추/0.정상.zip'), ('/content/01.데이터/1.Training/원천데이터/11.토마토/1.질병.zip', '01.데이터/1.Training/원천데이터/11.토마토/1.질병.zip'), ('/content/01.데이터/1.Training/원천데이터/05.상추/1.질병.zip', '01.데이터/1.Training/원천데이터/05.상추/1.질병.zip'), ('/content/01.데이터/1.Training/원천데이터/02.고추/1.질병.zip', '01.데이터/1.Training/원천데이터/02.고추/1.질병.zip')]
{'01.데이터/2.Validation/라벨링데이터/05.상추/0.정상.zip', '/content/Leaf-Disease-Detection-and-Classification/PlantDiseaseDetection/models

### AI Hub에서 API를 활용하여 데이터셋을 다운받기

#### aihubshell 실행파일 다운 후 실행권한 및 환경설정

In [ ]:
!curl -o "aihubshell" https://api.aihub.or.kr/api/aihubshell.do

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6634  100  6634    0     0   7285      0 --:--:-- --:--:-- --:--:--  7290


In [ ]:
!ls -al

total 16
drwxr-xr-x 2 root root 4096 May 20 01:18 .
drwxr-xr-x 1 root root 4096 May 20 01:06 ..
-rw-r--r-- 1 root root 6634 May 20 01:18 aihubshell


In [ ]:
!chmod +x aihubshell

In [ ]:
!sudo cp aihubshell /usr/bin
!cd /usr/bin/
!ls -al | grep aihubshell

-rwxr-xr-x 1 root root 6634 May 20 01:18 aihubshell


#### AIhubshell 명령어 사용
- aihubshell -help : 메뉴얼 출력
- aihubshell -mode { d | l }: 다운로드 모드와 list 모드가 있다
- -datasetkey (데이터셋 코드): AIhub에 있는 데이터셋의 코드를 입력하면 그 데이터셋에 접근할 수 있다.

In [ ]:
!aihubshell -help

aihubshell version 24.01.29 v0.3
aihubshell에 대한 명령어 및 옵션 상세


ENGL_CMGG	 KOREAN_CMGG			 DETAIL_CN
-aihubid  	 {사용자계정}	|	 https://aihub.or.kr 에서 회원가입 후 이용하실 수 있습니다.
						[mode d 를 사용할 시최초1회 입력 필수]
						안내 : 입력된 aihubid는 동일명으로 환경변수에 저장 합니다.
						예외 : console 재접속 또는 서버 재기동 초기화 될 수 있습니다.

-aihubpw  	 {사용자비밀번호}	|	 https://aihub.or.kr 에서 회원가입 후 이용하실 수 있습니다.
						[mode d 를 사용할 시최초1회 입력 필수]
						안내 : 입력된 aihubpw는 동일명으로 환경변수에 저장 합니다.
						예외 : console 재접속 또는 서버 재기동 초기화 될 수 있습니다.

-mode     	 l              	|	 [list 모드 안내]
						datasetkey 가 없는 경우 개방된 데이터셋의 제목과 key값을 제공합니다.
						datasetkey 를 입력한 경우 해당 데이터셋의 파일 목록을 아래와 같이 제공합니다.
						데이터셋의 파일목록, 용량, filekey를 제공합니다.
						공지사항이 있을 경우 안내합니다.

-mode     	 d              	|	 [download 모드 안내]
						[필수 이용조건] -aihubid, -aihubpw 최초 1회 입력 또는 별도 환경변수 추가
						[필수 파라미터] datasetkey
						filekey 입력한 경우 해당 파일을 다운로드 합니다.
						{111, 222, 333} 다중 파일 다운로드 지원
						filekey 옵션 누락시 전체 파일을 다운로드 합니다.
						[필수 사전 사항] https://aihub.or.kr 에서 데이터 사용신청 및 사용허가



In [ ]:
!aihubshell -mode l | grep "시설 작물"

153, 시설 작물 질병 진단 이미지
237, 시설 작물 개체 이미지


In [ ]:
!aihubshell -mode l -datasetkey 153

aihubshell version 24.01.29 v0.3
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─071.시설 작물 질병 진단
        ├─01.데이터
        │  ├─1.Training
        │  │  ├─라벨링데이터
        │  │  │  ├─01.가지
        │  │  │  │  ├─0.정상.zip | 15 MB | 46901
        │  │  │  │  ├─1.질병.zip | 394 KB | 46902
        │  │  │  │  └─9.증강.zip | 5 MB | 46903
        │  │  │  ├─02.고추
        │  │  │  │  ├─0.정상.zip | 3 MB | 46904
        │  │  │  │  ├─1.질병.zip | 1 MB | 46905
        │  │  │  │  └─9.증강.zip | 14 MB | 46906
        │  │  │  ├─03.단호박
        │  │  │  │  ├─0.정상.zip | 1 MB | 46814
        │  │  │  │  ├─1.질병.zip | 634 KB | 46815
        │  │  │  │  └─9.증강.zip | 8 MB | 46816
        │  │  │  ├─06.수박
        │  │  │  │  ├─9.증강.zip | 7 MB | 46825
        │  │  │  │  ├─1.질병.zip | 590 KB | 46824
        │  │  │  │  └─0

### 필요한 데이터만 선택 다운로드하기
- 선택하기 위해선 api키가 필요 -> 사이트에서 확인가능
- 접근 권한을 확인하기 위해 아이디 비번 필요

#### 환경 변수 저장
- 아이디, 비밀번호 저장

In [ ]:
aihub_id = '아이디를 입력하세요'
aihub_pw = '비밀번호를 입력하세요'

#### 훈련 데이터 셋 저장하기
  - 토마토
    - 원본데이터
      - 정상(19.88GB) : 46791
      - 질병(2.74GB): 46792
    - 라벨링 데이터
      - 정상(6.12MB) : 46838
      - 질병(907.61KB) : 46839

  - 상추
    - 원본데이터
      - 정상(15.34 GB):46767
      - 질병(6.32 GB) :46768
    - 라벨링 데이터
      - 정상(4.89 MB):46820
      - 질병(1.42 MB):46821

  - 고추
    - 원본데이터
      - 정상 (9.91 GB) :46849
      - 질병 (3.75 GB):46850
    - 라벨링 데이터
      - 정상 ( 3.37 MB) :46904
      - 질병 ( 1.11 MB) :46905


##### 토마토

In [ ]:
# 라벨 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46838 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 6268k    0 6268k    0     0  4547k      0 --:--:--  0:00:01 --:--:-- 13.5M
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/11.토마토/0.정상.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/11.토마토
병합이 완료 되었습니다. 


In [ ]:
# 라벨 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46839 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  910k    0  910k    0     0   811k      0 --:--:--  0:00:01 --:--:-- 1302k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/11.토마토/1.질병.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 1.질병.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/11.토마토
병합이 완료 되었습니다. 


In [ ]:
# 원본 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46791 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 19.8G    0 19.8G    0     0  15.3M      0 --:--:--  0:22:05 --:--:-- 8803k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part0
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part1073741824
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part2147483648
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part3221225472
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part4294967296
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part5368709120
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part6442450944
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part7516192768
071

In [ ]:
# 원본 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46792 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
** Resuming transfer from byte position 304906240
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part0
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토
^C


##### 상추

In [ ]:
# 라벨 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46820 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
** Resuming transfer from byte position 304906240
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토/0.정상.zip.part0
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/11.토마토
병합이 완료 되었습니다. 


In [ ]:
# 라벨 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46821 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1461k    0 1461k    0     0  1462k      0 --:--:-- --:--:-- --:--:-- 9951k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/05.상추/1.질병.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 1.질병.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/05.상추
병합이 완료 되었습니다. 


In [ ]:
# 원본 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46767 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 15.3G    0 15.3G    0     0  8886k      0 --:--:--  0:30:10 --:--:-- 9906k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part0
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part1073741824
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part2147483648
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part3221225472
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part4294967296
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part5368709120
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part6442450944
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part7516192768
071.시설_작물_질

In [ ]:
# 원본 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46768 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
** Resuming transfer from byte position 3231744
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추/0.정상.zip.part0
tar: Unexpected EOF in archive
tar: rmtlseek not stopped at a record boundary
tar: Error is not recoverable: exiting now
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/05.상추
병합이 완료 되었습니다. 


##### 고추

In [ ]:
# 라벨 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46904 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3458k    0 3458k    0     0  2607k      0 --:--:--  0:00:01 --:--:-- 6753k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/02.고추/0.정상.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/02.고추
병합이 완료 되었습니다. 


In [ ]:
# 라벨 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46905 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1134k    0 1134k    0     0  1130k      0 --:--:--  0:00:01 --:--:-- 6579k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/02.고추/1.질병.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 1.질병.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/라벨링데이터/02.고추
병합이 완료 되었습니다. 


In [ ]:
# 원본 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46849 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  9.9G    0  9.9G    0     0  5311k      0 --:--:--  0:32:36 --:--:-- 5718k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part0
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part1073741824
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part2147483648
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part3221225472
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part4294967296
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part5368709120
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part6442450944
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part7516192768
071.시설_작물_질

In [ ]:
# 원본 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46850 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
** Resuming transfer from byte position 345817088
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추/0.정상.zip.part0
tar: Unexpected EOF in archive
tar: rmtlseek not stopped at a record boundary
tar: Error is not recoverable: exiting now
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/1.Training/원천데이터/02.고추
병합이 완료 되었습니다. 


#### 검증 데이터 셋 저장하기
  - 토마토
    - 라벨링 데이터
      - 정상(2.01 MB):46735
      - 질병(113.45 KB):46736
    - 원천 데이터
      - 정상(6.96 GB):46678
      - 질병(364.14 MB):46679

  - 상추
    - 라벨링 데이터
      - 정상(68.06 KB):46811
      - 질병(182.40 KB):46812
    - 원천 데이터
      - 정상(182.40 MB):46753
      - 질병(828.02 MB):46754
      
  - 고추
    - 라벨링 데이터
      - 정상(19.08 KB):46802
      - 질병(428.33 KB):46803
    - 원천 데이터
      - 정상(48.83 MB):46744
      - 질병(1.39 GB): 46745




##### 토마토

In [ ]:
# 라벨 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46735 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
^C


In [ ]:
# 라벨 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46736 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  116k    0  116k    0     0   127k      0 --:--:-- --:--:-- --:--:--  127k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/11.토마토/1.질병.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 1.질병.zip in ./071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/11.토마토
병합이 완료 되었습니다. 


In [ ]:
# 원천 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46678 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 13.9M    0 13.9M    0     0  4217k      0 --:--:--  0:00:03 --:--:-- 4780k^C


In [ ]:
# 원천 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46679 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
** Resuming transfer from byte position 18780160
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/2.Validation/원천데이터/11.토마토/0.정상.zip.part0
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/2.Validation/원천데이터/11.토마토
^C


##### 상추

In [ ]:
# 라벨 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46811 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 72704    0 72704    0     0   110k      0 --:--:-- --:--:-- --:--:--  110k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/05.상추/0.정상.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/05.상추
병합이 완료 되었습니다. 


In [ ]:
# 라벨 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46812 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  185k    0  185k    0     0   228k      0 --:--:-- --:--:-- --:--:--  228k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/05.상추/1.질병.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 1.질병.zip in ./071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/05.상추
병합이 완료 되었습니다. 


In [ ]:
# 원본 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46753 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  111M    0  111M    0     0  6337k      0 --:--:--  0:00:18 --:--:-- 8348k^C


In [ ]:
# 원본 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46754 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
** Resuming transfer from byte position 120451072
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/2.Validation/원천데이터/05.상추/0.정상.zip.part0
tar: Unexpected EOF in archive
tar: rmtlseek not stopped at a record boundary
tar: Error is not recoverable: exiting now
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/2.Validation/원천데이터/05.상추
^C


##### 고추

In [ ]:
# 라벨 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46802 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 22528    0 22528    0     0  28288      0 --:--:-- --:--:-- --:--:-- 28288
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/02.고추/0.정상.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/02.고추
병합이 완료 되었습니다. 


In [ ]:
# 라벨 질병
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46803 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  431k    0  431k    0     0   291k      0 --:--:--  0:00:01 --:--:-- 2028k
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/02.고추/1.질병.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging 1.질병.zip in ./071.시설_작물_질병_진단/01.데이터/2.Validation/라벨링데이터/02.고추
병합이 완료 되었습니다. 


In [ ]:
# 원천 정상
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46744 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3392k    0 3392k    0     0  2721k      0 --:--:--  0:00:01 --:--:-- 2721k^C


In [ ]:
# 원천 질병
!aihubshell -mode d -datasetkey 153 -filekey 46745 -aihubid ditlswlwhs3@gmail.com -aihubpw dl994550!
command = f'!aihubshell -mode d -datasetkey 153 -filekey 46745 -aihubid {aihub_id} -aihubpw {aihub_pw}'
# 명령어 실행
get_ipython().system(command)

aihubshell version 24.01.29 v0.3
Authentication successful.
** Resuming transfer from byte position 120451072
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (33) HTTP server doesn't seem to support byte ranges. Cannot resume.
Request successful with HTTP status 200.
Download successful.
071.시설_작물_질병_진단/01.데이터/2.Validation/원천데이터/05.상추/0.정상.zip.part0
tar: Unexpected EOF in archive
tar: rmtlseek not stopped at a record boundary
tar: Error is not recoverable: exiting now
잠시 기다려 주세요 병합중 입니다. 
Merging 0.정상.zip in ./071.시설_작물_질병_진단/01.데이터/2.Validation/원천데이터/05.상추
병합이 완료 되었습니다. 
